## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append("../")
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"c:/Users/smong/Documents/Classwork/module_6_weatherpy/world_weather_analysis/Weather_Database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Pevek,RU,69.7008,170.3133,-25.64,98,100,8.61,overcast clouds
1,1,Avarua,CK,-21.2078,-159.7750,71.65,100,92,1.14,overcast clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,12.04,73,0,14.97,clear sky
3,3,New Norfolk,AU,-42.7826,147.0587,56.70,59,92,2.64,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,51.80,65,75,14.34,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Avarua,CK,-21.2078,-159.7750,71.65,100,92,1.14,overcast clouds
3,3,New Norfolk,AU,-42.7826,147.0587,56.70,59,92,2.64,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,51.80,65,75,14.34,broken clouds
5,5,Bredasdorp,ZA,-34.5322,20.0403,74.52,64,47,11.50,scattered clouds
6,6,Gat,IL,31.6100,34.7642,61.48,16,0,7.09,clear sky
7,7,Wharton,US,29.3116,-96.1027,61.92,100,40,6.91,scattered clouds
8,8,Port Alfred,ZA,-33.5906,26.8910,79.20,83,8,18.43,clear sky
10,10,Castro,BR,-24.7911,-50.0119,78.24,47,0,3.98,clear sky
11,11,Haverfordwest,GB,51.8017,-4.9691,50.05,94,100,10.00,overcast clouds
19,19,Mataura,NZ,-46.1927,168.8643,51.48,91,100,3.15,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df2 = preferred_cities_df.dropna()
preferred_cities_df2.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df2[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Avarua,CK,71.65,overcast clouds,-21.2078,-159.7750,
3,New Norfolk,AU,56.70,overcast clouds,-42.7826,147.0587,
4,Bluff,NZ,51.80,broken clouds,-46.6000,168.3333,
5,Bredasdorp,ZA,74.52,scattered clouds,-34.5322,20.0403,
6,Gat,IL,61.48,clear sky,31.6100,34.7642,
7,Wharton,US,61.92,scattered clouds,29.3116,-96.1027,
8,Port Alfred,ZA,79.20,clear sky,-33.5906,26.8910,
10,Castro,BR,78.24,clear sky,-24.7911,-50.0119,
11,Haverfordwest,GB,50.05,overcast clouds,51.8017,-4.9691,
19,Mataura,NZ,51.48,overcast clouds,-46.1927,168.8643,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
len(hotel_df.index)

292

In [21]:
# 7. Drop the rows where there is no Hotel Name.
nan = float("NaN")
hotel_df2 = hotel_df
hotel_df2.replace("", nan, inplace=True)
hotel_df2.dropna(subset = ["Hotel Name"], inplace=True)

In [22]:
len(hotel_df2.index)

273

In [25]:
# 8a. Create the output File (CSV)
output_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df2.to_csv(output_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df2[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))